In [5]:
import gensim.downloader as api
import gensim
from gensim.models import Phrases
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import ast

In [2]:
info = api.info()  # show info about available models/datasets

In [3]:
model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
#w2v = dict(zip(model.index2word, model.vectors))

In [6]:
filename = r'/home/miboj/NLP/document-summarizer/data/processed/articles.json'
file = open(filename, encoding='ascii', errors='ignore')
text = file.read()
file.close()

d = ast.literal_eval(text)

In [28]:
print(d['tokens'][0:10])
tokens_list = d['tokens']

[' Lt. Gen. David Allvin was confirmed by the Senate to be the Air Forces next vice chief of staff in a late-night vote Wednesday.', 'Allvins nomination to become vice chief and receive his fourth star was approved unanimously.', 'In a Thursday release, Chief of Staff Gen. Charles CQ Brown applauded Allvins confirmation.', 'I could not be more pleased with the confirmation of Gen. Allvin as the new vice chief of staff, Brown said.', 'When it comes to leading at the highest levels of joint strategy and policy, and as someone who sets the standard for critical collaboration with our allies and partners, there is no one more qualified for the role of vice chief.', 'Allvin will succeed the current vice chief, Gen. Seve Wilson, who is expected to retire after 39 years in uniform.', 'Wilson has served in his role since July 2016 and is the longest-serving vice chief in Air Force history.', 'Allvin now serves director of strategy, plans and policy for the Joint Staff at the Pentagon, and is a

In [37]:
import time
start_time = time.time()
words= []
stpwrds = stopwords.words('english') + list(string.punctuation) + ['—', '“', '”', "'", "’"]
for e, i in enumerate(tokens_list):
    a = nltk.word_tokenize(i)
    for word in a:
        if word not in stpwrds:
            words.append(word)
words[0][0:10]
print("--- %s seconds ---" % (time.time() - start_time))

--- 28.665889739990234 seconds ---


In [38]:
print(len(words))

1805595


In [39]:
model.train(words, total_examples=len(words), epochs=model.epochs)

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'train'

In [4]:
import re
def remove_empty_string(input_string):
    for e, i in enumerate(input_string):
        if i[-1] == ' ' and input_string[e+1][-1] == ' ':
            input_string[e] = i.rstrip()
    joined_string = ''.join(input_string)
    for e, i in enumerate(joined_string):
        if i == ' ' and joined_string[e+1] == ' ':
            del i
    sentences = nltk.sent_tokenize(joined_string)
    return sentences


In [5]:
raw_string = [" ROME — Defying reports that their planned partnership is ", "doomed to fail", ", France’s Naval Group and ", "Italy’s Fincantieri", " have announced a joint venture to build and export naval vessels. ", " The two ", "state-controlled shipyards", " said they were forming a 50-50 joint venture after months of talks to integrate their activities. The move comes as Europe’s fractured shipbuilding industry faces stiffer global competition. ", " The firms said in a statement that the deal would allow them to “jointly prepare winning offers for binational programs and export market,” as well as create joint supply chains, research and testing. ", " Naval Group and Fincantieri first announced talks on cooperation last year after the latter negotiated a controlling share in French shipyard STX. But the deal was reportedly losing momentum due to resistance from French industry and a political row between France and Italy over migrants. ", " The new deal falls short of the 10 percent share swap predicted by French Economy and Finance Minister Bruno Le Maire earlier this year, and far short of the total integration envisaged by Fincantieri CEO Giuseppe Bono. ", " The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.”", " Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.”", " The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.", "Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.", "By giving us your email, you are opting in to the Early Bird Brief.", " When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.”", " However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design. ", "Competition flares up for the follow-on portion of a deal previously won by the French shipbuilder.", " The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies. The work would include providing a common combat management system. ", " The statement was cautious about future acceleration toward integration. “A Government-to-Government Agreement would be needed to ensure the protection of sovereign assets, a fluid collaboration between the French and Italian teams and encourage further coherence of the National assistance programs, which provide a framework and support export sales,” the statement said.", " But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.”", " ", "Sebastian Sprenger", " in Paris contributed to this report."]
sentences = remove_empty_string(raw_string)

In [16]:
# The 'skipthoughts' module can be found at the root of the GitHub  repository linked above
#import skipthoughts

# You would need to download pre-trained models first
#model = skipthoughts.load_model()

#encoder = skipthoughts.Encoder(model)
#encoded =  encoder.encode(sentences)
a = model['ROME']
a.shape

(300,)

In [69]:
embeddings = []
stpwrds = stopwords.words('english') + list(string.punctuation) + ['—', '“', '”', "'", "’"]

for i in sentences:
    temp = []
    words = nltk.word_tokenize(i)
    for word in words:
        true_len = len(words) - len([w for w in stpwrds if w in words])
        #if word not in stpwrds:
        if word in model.vocab:
            v = model[word]/true_len
        else:
            v = np.zeros(300,)
        temp.append(v)
        a = sum(temp)
        np_temp = np.array(a)
    #embeddings.append(temp)
    embeddings.append(np_temp)
    
sentence_vectors = np.array(embeddings)
print(sentence_vectors.shape)
print(sentence_vectors[0])
print(sentence_vectors[0].shape)
print(len(sentence_vectors))

(15, 300)
[-4.10647199e-02  1.15833033e-01  4.68179430e-02  3.01566735e-02
 -4.26390265e-02 -4.14140113e-02  5.09112950e-03 -9.14737883e-02
  1.15961489e-01  7.05619480e-02 -5.54489263e-02 -5.17471970e-02
 -3.93617054e-02  3.07298626e-03 -1.31313161e-01  6.36556875e-02
  6.80143997e-03 -1.62658679e-02  6.40272156e-03  2.78074903e-03
 -3.21697548e-02  2.42880124e-03  4.88068952e-02 -8.09830372e-02
  1.16651453e-01 -6.45181352e-03  1.56701131e-02  8.51811966e-02
 -5.02133586e-02  3.33570529e-03  3.59264866e-02 -4.88997746e-02
 -1.36505791e-01 -5.90183365e-03  5.98376721e-02 -1.15436062e-02
 -1.11137075e-02 -1.40646266e-03 -4.94517435e-03  1.24216487e-02
  2.25936228e-02  2.69987690e-02  4.87063865e-02  1.12339185e-01
 -1.01066259e-02 -9.16615352e-02 -5.91802182e-02  6.87017024e-02
 -4.55587624e-02  1.04249041e-01 -7.18622771e-03  1.48351587e-01
  3.22079856e-02  2.01150548e-03 -4.05535478e-02  1.90111241e-02
 -1.46886411e-01 -1.08760668e-01  4.00569749e-02 -8.64456850e-02
 -2.33526807e-0

In [70]:
sim_mat = np.zeros([len(sentences), len(sentences)])

#for e, i in enumerate(sentence_vectors):
#    a = cosine_similarity(sentence_vectors[e])
sim_mat.shape    

15


(15, 15)

In [73]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [74]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [76]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [77]:
num_of_sentences = 4
for i in range(num_of_sentences):
    print(ranked_sentences[i][1])

The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.” Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.” The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.By giving us your email, you are opting in to the Early Bird Brief.
When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.” However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which 